###AWS Setup

In [ ]:
!pip install mlflow -q

!pip install aws configure
!pip install awscli
!aws configure

!pip install boto3

###Kaggle Setup

In [2]:
from google.colab import files
from pathlib import Path
import zipfile
import shutil

def setup():
    files.upload()
    src_dir = Path('/content/kaggle.json')
    dest_dir = Path('/root/.config/kaggle')
    dest_dir.mkdir(parents=True, exist_ok=True)
    shutil.move(str(src_dir), str(dest_dir))
    (dest_dir/'kaggle.json').chmod(0o600)

    !kaggle competitions download -c playground-series-s4e12

    zipfile.ZipFile('/content/playground-series-s4e12.zip').extractall()

setup()

Saving kaggle.json to kaggle.json
  0% 0.00/75.5M [00:00<?, ?B/s]
100% 75.5M/75.5M [00:00<00:00, 1.41GB/s]


In [3]:
import mlflow
import numpy as np
import pandas as pd

In [4]:
import boto3

In [5]:
from hyperopt import fmin, tpe, hp, STATUS_OK
from hyperopt.pyll import scope
import xgboost as xgb
from sklearn.metrics import root_mean_squared_error
import mlflow
import mlflow.xgboost

In [6]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [7]:
from mlflow import MlflowClient
from mlflow.entities import ViewType

In [8]:
MLFLOW_TRACKING_URI = 'http://ec2-56-228-36-113.eu-north-1.compute.amazonaws.com:5000'

### Preprocess data

In [9]:
def dump_pickle(obj, filename):
    with open(filename, 'wb') as f_out:
        pickle.dump(obj, f_out)


def prepare_data(path='/content/train.csv'):
    target = 'Premium Amount'
    features = ['Age', 'Annual Income', 'Number of Dependents',
                'Occupation', 'Credit Score', 'Property Type']

    df = pd.read_csv(path)
    df = df[features + [target]].copy()

    categorical = df.select_dtypes(include=['object']).columns.tolist()

    cat_col_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
    ])

    transformer = ColumnTransformer(transformers=[
        ('cat', cat_col_transformer, categorical)
    ], remainder='passthrough')

    transformed = transformer.fit_transform(df[features])

    with open('preprocessor.bin', 'wb') as f_out:
        pickle.dump(transformer, f_out)

    X_train, X_temp, y_train, y_temp = train_test_split(transformed, df[target], test_size=0.3)
    X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5)

    dump_pickle((X_train, y_train), 'train.pkl')
    dump_pickle((X_valid, y_valid), 'valid.pkl')
    dump_pickle((X_test, y_test), 'test.pkl')

prepare_data()

In [10]:
def load_file(file_path):
    with open(file_path, 'rb') as f_in:
        return pickle.load(f_in)

### Hyper parameter tuning

In [11]:
def hyper_parameter_tuning(num_trials=10):

    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    mlflow.set_experiment('XGBoost Tuning')

    X_train, y_train = load_file('train.pkl')
    X_valid, y_valid = load_file('valid.pkl')
    X_test, y_test = load_file('test.pkl')

    def objective(params):

        with mlflow.start_run():

            mlflow.log_params(params)

            model = xgb.XGBRegressor(**params, device='cuda')
            model.fit(X_train, y_train)

            y_pred = model.predict(X_valid)
            rmse = root_mean_squared_error(y_valid, y_pred)

            mlflow.log_metric('rmse', rmse)

            return {'loss': rmse, 'status': STATUS_OK}

    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 3, 15, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 100, 2000, 50)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -3, 1),
        'reg_lambda': hp.loguniform('reg_lambda', -3, 1)
    }

    best_result = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=num_trials,
    )

In [12]:
hyper_parameter_tuning()

2025/08/04 18:45:18 INFO mlflow.tracking.fluent: Experiment with name 'XGBoost Tuning' does not exist. Creating a new experiment.


  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:729: UserWarning: [18:45:49] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)



🏃 View run popular-shoat-257 at: http://ec2-56-228-36-113.eu-north-1.compute.amazonaws.com:5000/#/experiments/1/runs/a15eb48b794146958ac15060c6a326fc

🧪 View experiment at: http://ec2-56-228-36-113.eu-north-1.compute.amazonaws.com:5000/#/experiments/1

🏃 View run defiant-carp-995 at: http://ec2-56-228-36-113.eu-north-1.compute.amazonaws.com:5000/#/experiments/1/runs/7408a6b766ed482c9c5bd041186d70a8

🧪 View experiment at: http://ec2-56-228-36-113.eu-north-1.compute.amazonaws.com:5000/#/experiments/1

🏃 View run handsome-owl-461 at: http://ec2-56-228-36-113.eu-north-1.compute.amazonaws.com:5000/#/experiments/1/runs/ff9044c1d92b4cce91d561bc66b97cc7

🧪 View experiment at: http://ec2-56-228-36-113.eu-north-1.compute.amazonaws.com:5000/#/experiments/1

🏃 View run resilient-lynx-135 at: http://ec2-56-228-36-113.eu-north-1.compute.amazonaws.com:5000/#/experiments/1/runs/a5e2217714fd4d0e97ca40ff9656872e

🧪 View experiment at: http://ec2-56-228-36-113.eu-north-1.compute.amazonaws.com:5000/#/expe

### Model Registry

In [13]:
def register_model():

    HPO_EXPERIMENT_NAME = 'XGBoost Tuning'
    EXPERIMENT_NAME = 'XGB MODELS'

    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    mlflow.set_experiment(EXPERIMENT_NAME)

    def train_and_log_model(params):

        X_train, y_train = load_file('train.pkl')
        X_valid, y_valid = load_file('valid.pkl')
        X_test, y_test = load_file('test.pkl')

        with mlflow.start_run():
            parsed_params = {k: int(v) if v.isdigit() else float(v) for k, v in params.items()}

            model = xgb.XGBRegressor(**parsed_params, device='cuda')
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            test_rmse = root_mean_squared_error(y_test, y_pred)
            mlflow.log_metric("test_rmse", test_rmse)

            mlflow.log_artifact('preprocessor.bin')
            mlflow.xgboost.log_model(model, artifact_path='model')

    def run_register_model(top_n=3):

        client = MlflowClient()

        experiment = client.get_experiment_by_name(HPO_EXPERIMENT_NAME)
        runs = client.search_runs(
            experiment_ids=experiment.experiment_id,
            run_view_type=ViewType.ACTIVE_ONLY,
            max_results=top_n,
            order_by=["metrics.rmse ASC"]
        )
        for run in runs:
            train_and_log_model(run.data.params)

        experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
        best_run = client.search_runs(
            experiment_ids=experiment.experiment_id,
            order_by=['metrics.test_rmse ASC']
        )[0]

        mlflow.register_model(
            model_uri=f'runs:/{best_run.info.run_id}/model',
            name='BEST XGBOOST MODEL'
        )

    run_register_model()


In [14]:
register_model()

2025/08/04 18:49:43 INFO mlflow.tracking.fluent: Experiment with name 'XGB MODELS' does not exist. Creating a new experiment.
2025/08/04 18:50:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:1028: UserWarning: [18:50:17] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/08/04 18:50:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run zealous-bird-514 at: http://ec2-56-228-36-113.eu-north-1.compute.amazonaws.com:5000/#/experiments/2/runs/95dc2925694a4b248d37bd74a50fa3de
🧪 View experiment at: http://ec2-56-228-36-113.eu-north-1.compute.amazonaws.com:5000/#/experiments/2


2025/08/04 18:50:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:1028: UserWarning: [18:50:38] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/08/04 18:50:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run marvelous-pug-19 at: http://ec2-56-228-36-113.eu-north-1.compute.amazonaws.com:5000/#/experiments/2/runs/fe11aaa9631647c688bced0afeb94f89
🧪 View experiment at: http://ec2-56-228-36-113.eu-north-1.compute.amazonaws.com:5000/#/experiments/2


2025/08/04 18:50:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:1028: UserWarning: [18:51:05] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/08/04 18:51:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run unleashed-croc-11 at: http://ec2-56-228-36-113.eu-north-1.compute.amazonaws.com:5000/#/experiments/2/runs/ea8e0266d93b4e2a853ff73906935664
🧪 View experiment at: http://ec2-56-228-36-113.eu-north-1.compute.amazonaws.com:5000/#/experiments/2


Successfully registered model 'BEST XGBOOST MODEL'.
2025/08/04 18:51:23 WARNING mlflow.tracking._model_registry.fluent: Run with id 95dc2925694a4b248d37bd74a50fa3de has no artifacts at artifact path 'model', registering model based on models:/m-83cf6d47023347ce8ed1005ea991bacf instead
2025/08/04 18:51:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BEST XGBOOST MODEL, version 1
Created version '1' of model 'BEST XGBOOST MODEL'.
